In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from progressbar import ProgressBar
import time,os

In [1]:
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.updates import adam

Using gpu device 3: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 3007)


In [3]:
from Tars.vae.vae import VAE
from Tars.distribution import Gaussian,Bernoulli
from Tars.load_data import mnist

In [7]:
load,plot = mnist('../datasets/')
train_x,_,_,_,test_x,_ = load(test=True)

size      = (28,28)
n_x       = (28*28)
n_z       = 64
n_y       = 10

activation = rectify
rseed = 1234
np.random.seed(rseed)

n_epoch = 11
n_batch = 100

In [ ]:
q_in = InputLayer((None,n_x))
q_0  = DenseLayer(q_in,num_units=512,nonlinearity=activation)
q_1  = DenseLayer(q_0,num_units=512,nonlinearity=activation)
q_mean = DenseLayer(q_1,num_units=n_z,nonlinearity=linear)
q_var = DenseLayer(q_1,num_units=n_z,nonlinearity=softplus)
q = Gaussian(q_mean,q_var,given=[q_in])

p_in = InputLayer((None,n_z))
p_0  = DenseLayer(p_in,num_units=512,nonlinearity=activation)
p_1  = DenseLayer(p_0,num_units=512,nonlinearity=activation)
p_mean = DenseLayer(p_1,num_units=n_x,nonlinearity=sigmoid)
p = Bernoulli(p_mean,given=[p_in])

In [ ]:
model = VAE(q,p,n_batch,adam,l=1,random=rseed)

In [ ]:
n_sample = 100
sample_z  = np.random.standard_normal((n_batch, n_z)).astype(np.float32)

def plot_x_image(model,plot,z,t,i,n_sample):
    sample_x = model.p_sample_mean_x(z)
    fig = plt.figure(figsize=(10,10))
    X,cmap = plot(sample_x[:n_sample])

    for j,x in enumerate(X):
            ax = fig.add_subplot(10, 10, j + 1, xticks=[], yticks=[])
            ax.imshow(x,cmap)

    plt.savefig('../plot/%d/%04d.jpg'%(t,i))
    plt.close()

In [ ]:
t = int(time.time())
os.mkdir('../plot/%d' % t)

pbar = ProgressBar(maxval=n_epoch).start()
for i in range(n_epoch):
    np.random.shuffle(train_x)
    lowerbound_train = model.train([train_x])

    if i%10 == 0:
        log_likelihood_test = model.log_likelihood_test([test_x],k=1000,mode='iw')
        lw = "epoch = %d, lower bound (train) = %lf (%lf %lf) lower bound (test) = %lf\n" %(i,sum(lowerbound_train),lowerbound_train[0],lowerbound_train[1],np.mean(log_likelihood_test))
        f = open("../plot/%d/temp.txt" % t, "a")
        f.write(lw)
        f.close()
        print lw
        plot_x_image(model,plot,sample_z,t,i,n_sample)
        
    pbar.update(i)